In [ ]:
# aeg
# 10/14/22

# look at high temps at TE plants


In [1]:
import sys
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
# from pyPRMS.ParameterFile import ParameterFile
import os
# from PIL import Image
# import glob
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
#set up paths
seg_tave_water_fn = "C:/Users/galanter/OneDrive - DOI/1-Projects/TEWU/subset_2000.csv"
seg_map_dir = 'C:/Users/galanter/OneDrive - DOI/2-GIS/TE/segment_mapping_gf_v11'
out_dir = "C:Users/galanter/OneDrive - DOI/1-Projects/TEWU"
te_path = 'C:/WU/Thermoelectric/environmental/data'


In [3]:
print (sys.version)

3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 06:57:19) [MSC v.1929 64 bit (AMD64)]


In [4]:
# read in files
plant_list = pd.read_csv(os.path.join(te_path,'all_plants_all_years_1498.csv'))

In [5]:
seg_map_df = gpd.read_file(os.path.join(seg_map_dir, "segment_mapping_gf_v11.shp"))
seg_map_df = seg_map_df.astype({'nsegment_v': 'string'})

#merge the geodataframe with the plant list
plants_df_geo = seg_map_df.merge(plant_list, how = 'right', left_on = 'nsegment_v',
                                 right_on = 'nsegment_v11_new')

plants = plants_df_geo[['prms_id', 'eia_id']]
plants = plants[plants['prms_id'].notna()]

In [6]:
# Read the output csv file
fn = seg_tave_water_fn
print("reading", fn)

fp = open(fn) 
# header = fp.readline()
# colnames = header.split(',')
# colnames.pop(0)
# print(colnames)

ndates = 0
line = fp.readline()
toks = line.strip().split(',')
nfeatures = len(toks) - 1

while line:
    line = fp.readline()
    ndates += 1
fp.close()

print("ndates =", ndates, "nfeatures =", nfeatures)
# print(lncnt)
sim_dates = np.zeros(ndates, dtype='datetime64[D]')
sim_vals = np.zeros((ndates, nfeatures), dtype=float)

with open(fn) as fp:
    # Skip header
#     header = fp.readline()
    
    ii = 0
    Lines = fp.readlines()
    for line in Lines:
        foo = line.strip()
        toks = foo.split(',')
        sim_dates[ii] = toks[0]
        
        for jj in range(len(toks)-1):
            sim_vals[ii,jj] = float(toks[jj+1])
            
        ii += 1
        
print(sim_vals.shape)

reading C:/Users/galanter/OneDrive - DOI/1-Projects/TEWU/subset_2000.csv
ndates = 8036 nfeatures = 58489
(8036, 58489)


In [7]:
# eliminate excess columns
prms_list = plants_df_geo['prms_id'].dropna().astype(int).to_list()
prms_list.sort()


In [8]:
# make a dataframe of the sntemp data 
sntemp_data = sim_vals[:, prms_list]
sntemp_df = pd.DataFrame(sntemp_data, columns = prms_list)
# add the date column and filter for 2008- 2020
sntemp_df['date'] = sim_dates.tolist()
sntemp_df['month'] = sntemp_df['date']
sntemp_df['date'] = pd.to_datetime(sntemp_df['date'], format = '%Y-%m-%d')
#trim the fat
sntemp_df = sntemp_df.loc[(sntemp_df['date']> '2007-12-31')]
sntemp_df = sntemp_df.loc[(sntemp_df['date']< '2021-01-01')]

In [ ]:
print(seg_map)
segs = gpd.read_file(seg_map)
print(segs.shape)

In [ ]:
domain = gpd.read_file(domain_map)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(20,10))
domain.plot(ax=axs, color='#ffffff', edgecolor='#6a6a6a', linewidth=1)
segs.plot(linewidth=0.3, ax=axs)

In [ ]:
segs.head()

In [ ]:
segs_sort = segs.sort_values('model_idx', ascending=True)

In [ ]:
segs_sort.head()

In [ ]:
vmin = min(sim_vals.flatten())
vmax = max(sim_vals.flatten())
print(vmin, vmax)

# For plotting, set any temp < 0.1 to 0.1 because of the log transform
seg_tave_water_min = 0.1

sim_vals[sim_vals < seg_tave_water_min] = seg_tave_water_min

vmin = min(sim_vals.flatten())
vmax = max(sim_vals.flatten())
print(vmin, vmax)

In [ ]:
dates = pd.date_range('2020-01-02', '2021-12-31')
# dates = pd.date_range('2020-07-01', '2020-07-01')
ndates = len(dates)

# for idate in (range(0,ndates)):
if True:
    idate = 212
# for idate in (range(0,1)):
#     vmax = max(sim_vals.flatten())
    val_min = 0.1
    vals = sim_vals[idate, :]
    
    segs_sort["vals"] = vals
    
    fig, ax = plt.subplots(1, 1, figsize=(20,10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    ax = segs_sort.plot(column='vals', ax=ax, cmap='jet', linewidth=0.5,
        norm=matplotlib.colors.LogNorm(vmin=val_min, vmax=vmax), cax=cax,
        legend=True,)
        
    domain.geometry.boundary.plot(ax=ax, color=None, edgecolor='#6a6a6a', linewidth= 0.5)
    ax.set_axis_off()
    ax.set_title(str(dates[idate])[0:10] + " seg_tave_water (degrees C)" + " from NHM-AK v0.0.0", fontdict={'fontsize': 18, 'fontweight': 'medium'})
    plt.show()
    
    # Free memory
    fig = ax.get_figure()
    fig.clear()
    plt.close(fig)

In [ ]:
segs_sort.head()

In [ ]:
# sim_dates[ii]
# sim_vals[ii,jj]

In [ ]:
print(sim_dates)

In [ ]:
model_idx = 19006
temp_vals = sim_vals[:,model_idx-1]

plt.figure(figsize=(20,10))
plt.xlabel('Date')
plt.ylabel('Water Temperature, oC')
plt.plot(sim_dates, temp_vals)